In [89]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from sklearn.feature_extraction.text import CountVectorizer

# todo : https://www.doczamora.com/content-based-recommender-system-for-movies-with-tensorflow

In [3]:
df = pd.read_json('/Users/juliendefaut/Dev/Babelio/output/books-sample1.json', lines=True)

In [7]:
df.head()

,book_id,book_nb_comm,title,name,surname,tags,img_url,comm_id,user_id,note,date,appreciations,commentaire
0,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,"roman,romans policiers et polars,thriller,colo...",/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,2080694,706958,4.0,30 novembre 2019,85,Roadtrip désertique dans l'Ouest Américain en ...
1,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,"roman,romans policiers et polars,thriller,colo...",/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,1271915,319761,5.0,03 mars 2017,68,"De temps en temps, j'aime bien revenir vers l'..."
2,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,"roman,romans policiers et polars,thriller,colo...",/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,1093854,128814,5.0,27 juin 2016,67,"Depuis peu j'ai découvert grâce à "" Cardabelle..."
3,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,"roman,romans policiers et polars,thriller,colo...",/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,2711430,227017,4.5,22 juillet 2021,64,"Ils sont quatre. \nUne blonde , jeunette, venu..."
4,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,"roman,romans policiers et polars,thriller,colo...",/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,2685514,138438,4.5,26 juin 2021,64,Avez-vous entendu cette info effrayante ? « La...


In [10]:
df = df.drop_duplicates(subset='book_id')

In [12]:
df.head()

,book_id,book_nb_comm,title,name,surname,tags,img_url,comm_id,user_id,note,date,appreciations,commentaire
0,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,"roman,romans policiers et polars,thriller,colo...",/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,2080694,706958,4.0,30 novembre 2019,85,Roadtrip désertique dans l'Ouest Américain en ...
10,34397,37,Aucune bête aussi féroce,Edward,Bunker,"roman,autobiographie,romans policiers et polar...",/couv/CVT_cvt_Aucune-bete-aussi-feroce_5042.jpg,637101,16287,4.0,02 septembre 2014,65,"Le changement, c'est maintenant !\nC'est certa..."
107,927980,64,Celle qui voulait conduire le tram,Catherine,Cuenca,"roman,roman historique,historique,littérature ...",https://images-eu.ssl-images-amazon.com/images...,2019582,574706,4.5,22 septembre 2019,42,Un bon petit livre sur la situation des femmes...
117,869119,75,Les belles vies,Benoît,Minville,"roman d'apprentissage,roman,littérature jeunes...",/couv/CVT_Les-Belles-Vies_5033.jpg,1938765,5865,4.0,16 juin 2019,37,Vasco et Djib' habitent la banlieue parisienne...
241,1355375,38,Le Cercueil de Job,Lance,Weller,"roman étranger,roman,roman historique,histoire...",/couv/CVT_Le-cercueil-de-Job_9332.jpg,2775696,489894,5.0,24 septembre 2021,129,Rentrée littéraire 2021 # 28\nJe referme ce li...


In [78]:
df['combined'] = df['title'] + ' ' + df['name']+ ' ' + df['surname']
df['tag2'] = df['tags'].str.replace(","," ")

In [79]:
df = df.reset_index()

In [80]:
df.head()

,level_0,index,book_id,book_nb_comm,title,name,surname,tags,img_url,comm_id,user_id,note,date,appreciations,commentaire,combined,tag2
0,0,0,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,"roman,romans policiers et polars,thriller,colo...",/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,2080694,706958,4.0,30 novembre 2019,85,Roadtrip désertique dans l'Ouest Américain en ...,"Le gang de la clef à molette (Ne meurs pas, ô ...",roman romans policiers et polars thriller colo...
1,1,10,34397,37,Aucune bête aussi féroce,Edward,Bunker,"roman,autobiographie,romans policiers et polar...",/couv/CVT_cvt_Aucune-bete-aussi-feroce_5042.jpg,637101,16287,4.0,02 septembre 2014,65,"Le changement, c'est maintenant !\nC'est certa...",Aucune bête aussi féroce Edward Bunker,roman autobiographie romans policiers et polar...
2,2,107,927980,64,Celle qui voulait conduire le tram,Catherine,Cuenca,"roman,roman historique,historique,littérature ...",https://images-eu.ssl-images-amazon.com/images...,2019582,574706,4.5,22 septembre 2019,42,Un bon petit livre sur la situation des femmes...,Celle qui voulait conduire le tram Catherine C...,roman roman historique historique littérature ...
3,3,117,869119,75,Les belles vies,Benoît,Minville,"roman d'apprentissage,roman,littérature jeunes...",/couv/CVT_Les-Belles-Vies_5033.jpg,1938765,5865,4.0,16 juin 2019,37,Vasco et Djib' habitent la banlieue parisienne...,Les belles vies Benoît Minville,roman d'apprentissage roman littérature jeunes...
4,4,241,1355375,38,Le Cercueil de Job,Lance,Weller,"roman étranger,roman,roman historique,histoire...",/couv/CVT_Le-cercueil-de-Job_9332.jpg,2775696,489894,5.0,24 septembre 2021,129,Rentrée littéraire 2021 # 28\nJe referme ce li...,Le Cercueil de Job Lance Weller,roman étranger roman roman historique histoire...


In [71]:
df.query('book_id == 885171')

,index,book_id,book_nb_comm,title,name,surname,tags,img_url,comm_id,user_id,note,date,appreciations,commentaire,combined
759,42647,885171,57,"La part des ombres, tome 1",Gabriel,Katz,"saga,roman,fantastique,fantasy,littérature jeu...",https://images-eu.ssl-images-amazon.com/images...,1600968,445500,4.0,26 avril 2018,73,Le lecteur a plaisir à retrouver dans ce roman...,"La part des ombres, tome 1 Gabriel Katz"


In [81]:
cm = CountVectorizer().fit_transform(df['tag2'])

In [73]:
cm

<1102x3293 sparse matrix of type '<class 'numpy.int64'>'
	with 6731 stored elements in Compressed Sparse Row format>

In [94]:
cs = cosine_similarity(cm)

In [95]:
def reco_book(book_id, max_book=5):
    index_book = df.query('book_id == @book_id').index.values.astype(int)[0]
    cs_book = cs[index_book]
    scores = [(i,bi) for i,bi in enumerate(cs_book)]
    sorted_scores = sorted(scores, key=lambda x:x[1], reverse=True)
    sorted_scores = sorted_scores[1:max_book+1]
    reco = [(i, df.iloc[i,:]['book_id'],df.iloc[i,:]['title']) for i,s in sorted_scores]
    return reco


In [99]:
reco_book(927980) # Celle qui voulait conduire le tram

[(1078, 111122, 'Parvana : Une enfance en Afghanistan'),
 (945, 1184143, 'Le Réseau Alice'),
 (850, 531546, 'Moi, Malala'),
 (202, 1244887, 'La femme qui en savait trop'),
 (11, 992430, '28 jours')]

In [97]:
reco_book(885171) # La part des ombres, tome 1

[(217, 8676, 'Le Dernier Elfe'),
 (140, 243528, "A comme Association, Tome 5 : Là où les mots n'existent pas"),
 (1020, 7718, 'Le livre des étoiles, tome 1 : Qadehar le Sorcier'),
 (517, 1109831, 'La Légende des quatre, tome 3 : Le Clan des Serpents'),
 (1070, 1360868, "La passeuse de mots, tome 2 : L'oeil de vérité")]